## Generate Static Graphs

These are the input parameters for the notebook. They will be automatically changed when the scripts to generate monthly statistics are run. You can modify them manually to generate multiple plots locally as well.

Pass in `None` to remove the filters and plot all data. This is not recommended for production settings, but might be useful for reports based on data snapshots.

In [ ]:
year = 2020
month = 11
program = "default"
study_type = "study"
mode_of_interest = None
include_test_users = False
use_imperial = False
sensed_algo_prefix = "cleaned"
bluetooth_only = False

In [ ]:
from collections import defaultdict

import numpy as np
import pandas as pd

from plots import *
import scaffolding

sns.set_style("whitegrid")
sns.set()
%matplotlib inline

# get metric vs imperial vars
label_units, short_label, label_units_lower, distance_col, weight_unit = scaffolding.get_units(use_imperial)

## Collect Data From Database

In [ ]:
expanded_ct, file_suffix, quality_text, debug_df = scaffolding.load_viz_notebook_sensor_inference_data(year,
                                                                            month,
                                                                            program,
                                                                            include_test_users,
                                                                            sensed_algo_prefix)

In [ ]:
#if fleet, replace primary_mode with primary_ble_sensed mode
try:
    if bluetooth_only and 'ble_sensed_summary' in expanded_ct.columns:
        expanded_ct = expanded_ct[expanded_ct['ble_sensed_summary'].notna()]
        print(expanded_ct.ble_sensed_summary.head())
        expanded_ct["primary_mode"] = expanded_ct.ble_sensed_summary.apply(lambda md: max(md["distance"], key=md["distance"].get))
        unique_keys = expanded_ct.groupby("primary_mode").agg({distance_col: "count"}).index
        print(unique_keys)
        colors_mode, colors_purpose, colors_sensed = scaffolding.mapping_color_labels({},  dict(zip(unique_keys, unique_keys)), {})
        colors_sensed = colors_mode
except ValueError as e:
    print("Got ValueError ", e)

## Generic Metrics

### Average miles per transport mode selected (primary_mode)

In [ ]:
plot_title_no_quality=" Average Miles for each mode with > 3 entries\n(inferred by OpenPATH from phone sensors)"
file_name ='average_miles_sensed_mode%s' % file_suffix

try:
    dist = expanded_ct.groupby('primary_mode').agg({distance_col: ['sum', 'count' , 'mean']})
    dist.columns = ['Total ('+label_units_lower+')', 'Count', 'Average ('+label_units_lower+')']
    dist = dist.reset_index()
    dist =dist.sort_values(by=['Total ('+label_units_lower+')'], ascending=False)
    
    data = dist.drop((dist.query("Count < 3").index)).sort_values(by=['Average ('+label_units_lower+')'], ascending=False)
    x='primary_mode'
    y='Average ('+label_units_lower+')'
    
    plot_title= plot_title_no_quality+"\n"+quality_text
    
    barplot_mode(data,x,y,plot_title, expanded_ct['primary_mode'].dropna().unique().tolist(), file_name)
    alt_text = store_alt_text_bar(pd.DataFrame(data['Average ('+label_units_lower+')'].values, data['primary_mode']), file_name, plot_title)
except:
    generate_missing_plot(plot_title_no_quality,debug_df,file_name)
    alt_text = store_alt_text_missing(debug_df, file_name, plot_title_no_quality) 

### Number of trips by day¶

In [ ]:
plot_title_no_quality="Number of trips by day\n(inferred by OpenPATH from phone sensors)"
file_name ='ntrips_sensed_per_day%s' % file_suffix

try:
    fq_days = expanded_ct.groupby(['start_local_dt_day']).agg({'start_local_dt_day': ['sum', 'count']})
    fq_days = fq_days.reset_index()
    fq_days.columns = ['Day of the Month', 'Total', 'Number of Trips']

    data = fq_days
    x = 'Day of the Month'
    y = 'Number of Trips'
    
    plot_title= plot_title_no_quality+"\n"+quality_text

    barplot_day(data,x,y,plot_title,file_name)
    alt_text = store_alt_text_bar(pd.DataFrame(data['Number of Trips'].values, data['Day of the Month']), file_name, plot_title)
except:
    generate_missing_plot(plot_title_no_quality,debug_df,file_name)
    alt_text = store_alt_text_missing(debug_df, file_name, plot_title_no_quality)    

### Number of trips by day of week¶

In [ ]:
plot_title_no_quality="Number of trips by weekday\n(inferred by OpenPATH from phone sensors)"
file_name ='ntrips_sensed_per_weekday%s' % file_suffix
try:
    fq_weekdays = expanded_ct.groupby(['start_local_dt_weekday']).agg({'start_local_dt_weekday': ['sum', 'count']})
    fq_weekdays = fq_weekdays.reset_index()
    fq_weekdays.columns = ['Weekday', 'Total', 'Number of Trips']
    weekday_labels = ["Mon", "Tue", "Wed", "Thu", "Fri", "Sat", "Sun"]
    fq_weekdays["Weekday"] = fq_weekdays.Weekday.apply(lambda x: weekday_labels[x])

    data = fq_weekdays
    x = 'Weekday'
    y = 'Number of Trips'

    plot_title= plot_title_no_quality+"\n"+quality_text
    
    barplot_day(data,x,y,plot_title,file_name)
    alt_text = store_alt_text_bar(pd.DataFrame(data['Number of Trips'].values, data['Weekday']), file_name, plot_title)
except:
    generate_missing_plot(plot_title_no_quality,debug_df,file_name)
    alt_text = store_alt_text_missing(debug_df, file_name, plot_title_no_quality)    